# Download The Dataset

In [ ]:
!pip install datasets

In [ ]:
import torch

In [ ]:

device='cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [ ]:
!pip install --quiet kaggle

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail
License(s): CC0-1.0
 99% 498M/503M [00:02<00:00, 124MB/s] 
100% 503M/503M [00:03<00:00, 175MB/s]


In [ ]:
!kaggle datasets download -d cuitengfeui/textsummarization-data

Dataset URL: https://www.kaggle.com/datasets/cuitengfeui/textsummarization-data
License(s): unknown
  0% 0.00/10.3M [00:00<?, ?B/s]
100% 10.3M/10.3M [00:00<00:00, 772MB/s]


In [ ]:
!kaggle datasets download -d marawanxmamdouh/dialogsum

Dataset URL: https://www.kaggle.com/datasets/marawanxmamdouh/dialogsum
License(s): other
  0% 0.00/8.05M [00:00<?, ?B/s]
100% 8.05M/8.05M [00:00<00:00, 253MB/s]


In [ ]:
!unzip newspaper-text-summarization-cnn-dailymail.zip -d text_summerisation

Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: text_summerisation/cnn_dailymail/test.csv  
  inflating: text_summerisation/cnn_dailymail/train.csv  
  inflating: text_summerisation/cnn_dailymail/validation.csv  


In [ ]:
!unzip textsummarization-data.zip -d summery

Archive:  textsummarization-data.zip
  inflating: summery/test.csv        
  inflating: summery/train.csv       
  inflating: summery/train_ds.csv    
  inflating: summery/val.csv         
  inflating: summery/valid_ds.csv    


In [ ]:
!unzip dialogsum.zip -d dialog

Archive:  dialogsum.zip
  inflating: dialog/CSV/hiddentest_dialogue.csv  
  inflating: dialog/CSV/hiddentest_topic.csv  
  inflating: dialog/CSV/test.csv     
  inflating: dialog/CSV/train.csv    
  inflating: dialog/CSV/validation.csv  
  inflating: dialog/JSONL/hiddentest_dialogue.jsonl  
  inflating: dialog/JSONL/hiddentest_topic.jsonl  
  inflating: dialog/JSONL/test.jsonl  
  inflating: dialog/JSONL/train.jsonl  
  inflating: dialog/JSONL/validation.jsonl  


# Data Preprocessing

In [ ]:
import pandas as pd

In [ ]:
train_data_cnn=pd.read_csv('/content/text_summerisation/cnn_dailymail/train.csv')

In [ ]:
train_data_summary_1=pd.read_csv('/content/summery/train.csv')

In [ ]:
train_data_summary_2=pd.read_csv('/content/summery/train_ds.csv')

In [ ]:
train_data_summary_2=train_data_summary_2.rename(columns={
    'article':'document',
    'title':'summary'
})

In [ ]:
train_data_summary=pd.concat([train_data_summary_1,train_data_summary_2],ignore_index=True)

In [ ]:
train_data_cnn=train_data_cnn.drop(['id'],axis=1)

In [ ]:
train_data_dialog=pd.read_csv('/content/dialog/CSV/train.csv')

In [ ]:
train_data_dialog=train_data_dialog.drop(['id','topic'],axis=1)

In [ ]:
train_data_dialog=train_data_dialog[:10000]

In [ ]:
import re

def remove_person_tags(text):
    return re.sub(r'#Person\d+#', '', text)

train_data_dialog['dialogue'] = train_data_dialog['dialogue'].apply(remove_person_tags)
train_data_dialog['summary'] = train_data_dialog['summary'].apply(remove_person_tags)

# Save the cleaned file (optional)
train_data_dialog.to_csv('train_dialogue.csv', index=False)

In [ ]:
train_data_dialog=pd.read_csv('/content/train_dialogue.csv')

In [ ]:
train_data_dialog.head()

,dialogue,summary
0,": Hi, Mr. Smith. I'm Doctor Hawkins. Why are y...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,": Hello Mrs. Parker, how have you been?\n: Hel...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,": Excuse me, did you see a set of keys?\n: Wha...",'s looking for a set of keys and asks for 's h...
3,: Why didn't you tell me you had a girlfriend?...,'s angry because didn't tell that had a gir...
4,": Watsup, ladies! Y'll looking'fine tonight. M...",Malik invites Nikki to dance. Nikki agrees if ...


In [ ]:
train_data_cnn = train_data_cnn.sample(n=30000, random_state=42)

In [ ]:
train_data_cnn=train_data_cnn.rename(columns={
    # 'article':'document',
    'highlights':'summary'
})

In [ ]:
train_data_summary.head()

,document,summary
0,jason blake of the islanders will miss the res...,blake missing rest of season
1,the u.s. military on wednesday captured a wife...,u.s. arrests wife and daughter of saddam deput...
2,craig bellamy 's future at west ham appeared i...,west ham drops bellamy amid transfer turmoil
3,cambridge - when barack obama sought advice be...,in search for expertise harvard looms large
4,"wall street held on to steep gains on monday ,...",wall street ends a three-day losing streak


In [ ]:
train_data_cnn.iloc[890]['summary']

"The daily transcript is a written version of each day's CNN Student News program .\nUse this transcript to help students with reading comprehension and vocabulary .\nUse the weekly Newsquiz to test your knowledge of stories you saw on CNN Student News ."

In [ ]:
train_data_dialog=train_data_dialog.rename(columns={
    'dialogue':'article',
    'summary':'summary'
})

In [ ]:
train_data_dialog['article'][87]

": Can you tell me about some good deals on produce? \n: The mangoes are on sale today. \n: What exactly are mangoes? \n: They have yellowish red skin. It's a fruit with one big seed. \n: Is the seed edible? \n: Maybe if you were a parrot. I wouldn't recommend it. \n: How much does a mango cost? \n: Normally, they're $2 each. Today, they're only $1 each. \n: Maybe I won't like the taste. \n: It's hard to describe. They're sweet, but also sort of citrusy. \n: How do I tell the difference between a ripe one and an unripe one? \n: They're similar to an avocado. When the outside feels soft, they're ripe. \n: Where do most of them come from? \n: These are from Mexico. "

In [ ]:
train_data_summary=train_data_summary.rename(columns={
    'document':'article',
    'article':'summary'
})

In [ ]:
train_data=pd.concat([train_data_summary,train_data_cnn,train_data_dialog],ignore_index=True)

In [ ]:
train_data=train_data.drop_duplicates()

In [ ]:
for column in train_data.columns:
    if train_data[column].dtype == 'object':  # Only process string columns
        train_data[column] =train_data[column].str.replace('#', '', regex=False).str.replace('\n', ' ', regex=False)

In [ ]:
train_data = train_data.sample(n=100000, random_state=42)

In [ ]:
train_data_cnn.head()

,article,summary
272581,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...
772,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...
171868,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...
63167,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...
68522,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...


In [ ]:
torch.cuda.empty_cache()  # Clears unused memory

In [ ]:
def to_lowercase(text):
    if pd.isna(text):
        return ''
    return text.lower()

for col in ['article', 'summary']:
    train_data[col] = train_data[col].apply(remove_person_tags).apply(to_lowercase)

# Save cleaned file
train_data.to_csv('train_dataset.csv', index=False)


In [ ]:
import re

def remove_punctuation(text):
    if pd.isna(text):
        return ''
    return re.sub(r'[^\w\s]', '', text)  # Keep only words and spaces

train_data['dialogue'] = train_data['article'].apply(remove_punctuation)
train_data['summary'] = train_data['summary'].apply(remove_punctuation)


In [ ]:
train_data=pd.read_csv('/content/train_dataset.csv')

In [ ]:
train_data.head()

,article,summary
0,`` perfect dark '' is a sequel of sorts to `` ...,perfect dark fun but expect action rather than...
1,pakistani president gen. pervez musharraf said...,pakistani leader says closer ties with russia ...
2,new zealand foreign minister murray mccully on...,new zealand calls for immediate ceasefire in s...
3,"by . emma reynolds . published: . 11:28 est, 1...",the 11 relatives locked themselves in their tu...
4,former spice girl melanie brown has given birt...,former spice girl melanie brown gives birth to...


In [ ]:
train_data['article'][87566]

'zurich, switzerland (cnn) -- top international soccer officials are meeting thursday in switzerland to discuss whether to change the dates for the 2022 world cup tournament in qatar from summer to winter amid concerns over heat. the two-day meeting of fifa\'s executive committee in zurich will also consider allegations that migrant workers employed on huge world cup-related construction projects are being abused. dozens of workers\' rights protesters, waving flags and brandishing red cards, gathered outside the complex ahead of the meeting as they delivered a letter to fifa president sepp blatter calling on soccer\'s world governing body to ensure fair labor practices. a fifa official took the letter inside, saying that fifa is taking the matter seriously. members of the executive committee, known as exco, declined to answer reporters\' questions as they entered. in addition to the issues of qatar\'s summer heat and concerns over labor conditions, the committee is expected to hear an 

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

#Load the dataset and train the model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
# Set legacy=False to opt into the new tokenizer behavior
tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

# # Convert Pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
train_dataset = train_dataset.shuffle(seed=42)

In [ ]:
train_dataset

Dataset({
    features: ['article', 'summary'],
    num_rows: 100000
})

In [ ]:
def preprocess_function(examples):
    # Add "summarize:" prefix for T5
    inputs = ["summarize: " + text for text in examples["article"]]
    targets = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=150,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=1000,
    load_from_cache_file=False
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

train_dataset = train_dataset.shuffle(seed=42)
split_dataset = train_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/summarization_checkpoints",
    eval_strategy="epoch",
    learning_rate=0.01,
    per_device_train_batch_size=32,  # Reduced for typical GPU memory
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_total_limit=7,
    num_train_epochs=7,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,           # Provide your evaluation dataset here
    tokenizer=tokenizer,                 # Deprecated: future versions will require processing_class instead
)

trainer.train()

<ipython-input-53-7d5af2f16dd4>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.608300,0.483524
2,0.491000,0.459607
3,0.461500,0.449637
4,0.421300,0.441507
5,0.392400,0.434186
6,0.373500,0.432253


Epoch,Training Loss,Validation Loss
1,0.608300,0.483524
2,0.491000,0.459607
3,0.461500,0.449637
4,0.421300,0.441507
5,0.392400,0.434186
6,0.373500,0.432253
7,0.354900,0.434360


TrainOutput(global_step=4431, training_loss=0.43278031294299046, metrics={'train_runtime': 12410.8895, 'train_samples_per_second': 45.686, 'train_steps_per_second': 0.357, 'total_flos': 7.6738801434624e+16, 'train_loss': 0.43278031294299046, 'epoch': 7.0})

#save an test the model

In [ ]:
import shutil

shutil.rmtree('/content/drive', ignore_errors=True)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_directory = "/content/drive/MyDrive/saved_model_summary"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/saved_model_summary/tokenizer_config.json',
 '/content/drive/MyDrive/saved_model_summary/special_tokens_map.json',
 '/content/drive/MyDrive/saved_model_summary/spiece.model',
 '/content/drive/MyDrive/saved_model_summary/added_tokens.json')

In [ ]:
def generate_summary(article_text):
    input_text = "summarize: " + article_text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512)
    summary_ids = model.generate(input_ids.to(device), max_length=500, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage:
sample_article = """
Dear all,

We’ve received an exciting internship opportunity from Number7 AI, a company founded by an alumnus of IIT Indore. Number7 AI is an AI-first company with the vision to automate every mediocre business task by revolutionising how businesses deal with documents. Businesses deal with a number of documents (in volumes and types) in various business processes. Every business process has a logic meaning it can be automated using AI with recent innovations in AI Agents and Agentic Architecture.

The company is being led by 3 people right now, each with the expertise who together will commercialise the vision of the company.
Vishal Rajput (Linkedin, Blog) - The Visionary who brings in the latest innovations of AI into our products with updates as new as after a week of launch. Currently working in Belgium in a cutting-edge drone company that operates with various governments and militaries in Europe.
Venkataraman Subramanian (LinkedIn) - The Architect behind Vishal's vision and bringing the vision to life through developing the product with the experience of developing advanced robotics automations. Currently Working in Munich, Germany.
Naveen George (LinkedIn) - The Strategist acting as the bridge between Technology and Business, recognising opportunities of how we want to get into the market and diversify. Currently Working as a Business Analyst in London, UK.
What Interns Can Expect:

Collaborate with a Truly international team spanning different countries.
Learn about the latest innovations in AI: As AI continues to evolve, it is important to understand how business professionals provide value alongside AI.

Opportunity to continue long-term based on performance.

Hands-on exposure to frameworks like Go-To-Market Strategy and Product-Market Fit (especially valuable for those considering an MBA)

What They Expect from Interns:
Use AI Tools: In today's era, there are those who use AI and those who'll be replaced by it. We want you to learn and use AI tools because it'll bring out what values you can provide as an individual.

Mindset to learn: Being a student, there's a lot you'll have to learn. We don't expect you to be knowledgeable from the start, but your goal should be to become the Subject Matter Expert. This will help you in the long-run.

Ask genuine questions: No one knows everything from the start. The aim is to put the effort into answering it yourself first and get in touch if you feel stuck. It engages critical thinking.

Introductory Call Details:
The first call for all interested students will be held:
Date: Thursday, 15th February
Time: 10:00 PM
Agenda: Company overview, internship expectations, selection process, and problem statement briefing

The founder will share an assignment along with resources. You’ll be expected to submit your version of the solution, based on which a few interns will be selected to work closely with the team.

Regards,
"""
print(generate_summary(sample_article))

number7 AI is an AI-first company with the vision to automate every mediocre business task by revolutionising how businesses deal with documents. the visionary who brings in the latest innovations of AI into their products with updates as new as after a week of launch.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
